In [1]:
import os
from ortools.sat.python import cp_model
import pandas as pd

In [6]:
os.chdir("..")

# Load data from Excel files
raeume_df = pd.read_csv('data/raeume.csv')
veranstaltungen_df = pd.read_csv('data/veranstaltungen.csv', parse_dates=["datum_von", "datum_bis"])

print("Data loaded successfully!")
print(f"Rooms loaded: {len(raeume_df)}")
print(f"Events to schedule: {len(veranstaltungen_df)}")

Data loaded successfully!
Rooms loaded: 29
Events to schedule: 20


In [7]:
display(raeume_df.head())
display(raeume_df.describe())

,raum_id,raum_plaetze,raum_klausur_plaetze,veranstaltungsort
0,1,28,14,A
1,2,30,15,A
2,3,98,49,A
3,4,72,36,A
4,5,40,20,A


,raum_id,raum_plaetze,raum_klausur_plaetze
count,29.000000,29.000000,29.000000
mean,15.000000,58.931034,24.275862
std,8.514693,58.813222,18.769079
min,1.000000,0.000000,0.000000
25%,8.000000,28.000000,10.000000
50%,15.000000,40.000000,20.000000
75%,22.000000,72.000000,36.000000
max,29.000000,300.000000,72.000000


In [8]:
display(veranstaltungen_df.head())
veranstaltungen_df['anzahl_teilnehmer'].describe()

,veranstaltung_hash_id,kurs_hash_id,datum_von,datum_bis,lehrgang_ids,anzahl_teilnehmer
0,0096055fb3a62972cce6cbdf650358ab14b0a6c38e5667...,07bece75facc7c388b34a2d393cbec8ff453eb4f90446e...,2025-05-23 12:30:00,2025-05-23 15:45:00,[1],9
1,01bc374a17c881a80bcdb24c71d9497110b4a4ff045d87...,d4972ad89e291495520cdd0604d096a181a9013c9b93c3...,2025-05-23 08:30:00,2025-05-23 11:45:00,"[2, 3]",49
2,03d66985fadc20d6a89f13652a8377064179c98a69b7f5...,e2ab5a7db0069b43b968f164bf16c86394ca616590676c...,2025-05-23 18:00:00,2025-05-23 21:15:00,[4],23
3,0d8e3d38c4fc6f6050d4cedbbf8e4f678740fd3de585ab...,e5a0d08839066813d987c727ca09aa196af70ddc78f67f...,2025-05-23 18:00:00,2025-05-23 19:30:00,[5],6
4,104140207126fb550cb8cdc5d7cefe479994328987685a...,432c8dfafa45b1a53d619ceaf2ab9e54a9f995144f27d6...,2025-05-23 12:30:00,2025-05-23 15:45:00,[6],9


count    20.000000
mean     25.400000
std      18.715038
min       6.000000
25%       9.000000
50%      19.500000
75%      40.750000
max      64.000000
Name: anzahl_teilnehmer, dtype: float64

## Next: Room Assignment Optimization

We will now formulate and solve the room assignment problem, ensuring:
- Room capacity is sufficient for each event
- No double-booking (respect existing bookings and avoid overlaps)
- Each event is assigned to at most one room

We will use a simple greedy or optimization approach in the next steps.

In [9]:
# Create the optimization model
model = cp_model.CpModel()

# Get dimensions
num_events = len(veranstaltungen_df)
num_rooms = len(raeume_df)

# Decision variables
# event_room_assignment[event_idx, room_idx] = 1 if event is assigned to room, 0 otherwise
event_room_assignment = {}
for event_idx in range(num_events):
    for room_idx in range(num_rooms):
        event_room_assignment[event_idx, room_idx] = model.NewBoolVar(
            f'event_{event_idx}_room_{room_idx}_assignment'
        )

# room_used[room_idx] = 1 if room is used at all, 0 otherwise
room_used = {}
for room_idx in range(num_rooms):
    room_used[room_idx] = model.NewBoolVar(f'room_{room_idx}_used')

# Helper function to check if two time periods overlap
def time_periods_overlap(start1, end1, start2, end2):
    return start1 < end2 and start2 < end1

# Helper function to check if two events are from the same lehrgang
def same_lehrgang(event1_ids, event2_ids):
    return bool(set(eval(event1_ids)) & set(eval(event2_ids)))

# Constraints

# 1. Each event must be assigned to exactly one room
for event_idx in range(num_events):
    model.Add(sum(event_room_assignment[event_idx, room_idx] 
                  for room_idx in range(num_rooms)) == 1)

# 2. Room capacity must exceed number of participants
for event_idx in range(num_events):
    event_participants = veranstaltungen_df.iloc[event_idx]['anzahl_teilnehmer']
    for room_idx in range(num_rooms):
        room_capacity = raeume_df.iloc[room_idx]['raum_plaetze']
        if room_capacity < event_participants:
            model.Add(event_room_assignment[event_idx, room_idx] == 0)

# 3. No double booking (respect existing bookings and avoid overlaps)
for event1_idx in range(num_events):
    for event2_idx in range(event1_idx + 1, num_events):
        if time_periods_overlap(
            veranstaltungen_df.iloc[event1_idx]['datum_von'],
            veranstaltungen_df.iloc[event1_idx]['datum_bis'],
            veranstaltungen_df.iloc[event2_idx]['datum_von'],
            veranstaltungen_df.iloc[event2_idx]['datum_bis']
        ):
            for room_idx in range(num_rooms):
                model.Add(event_room_assignment[event1_idx, room_idx] + 
                         event_room_assignment[event2_idx, room_idx] <= 1)

# 4. Same lehrgang events must be in the same room on the same day
for event1_idx in range(num_events):
    for event2_idx in range(event1_idx + 1, num_events):
        if (same_lehrgang(veranstaltungen_df.iloc[event1_idx]['lehrgang_ids'], 
                         veranstaltungen_df.iloc[event2_idx]['lehrgang_ids']) and
            veranstaltungen_df.iloc[event1_idx]['datum_von'].date() == 
            veranstaltungen_df.iloc[event2_idx]['datum_von'].date()):
            for room_idx in range(num_rooms):
                model.Add(event_room_assignment[event1_idx, room_idx] == 
                         event_room_assignment[event2_idx, room_idx])

# 5. Link event_room_assignment and room_used variables
for room_idx in range(num_rooms):
    for event_idx in range(num_events):
        model.Add(event_room_assignment[event_idx, room_idx] <= room_used[room_idx])

# Objective: Minimize the sum of:
# 1. Difference between capacity and participants (weighted by 1)
# 2. Number of rooms used (weighted by 1000 to prioritize)
capacity_differences = []
for event_idx in range(num_events):
    event_participants = veranstaltungen_df.iloc[event_idx]['anzahl_teilnehmer']
    for room_idx in range(num_rooms):
        room_capacity = raeume_df.iloc[room_idx]['raum_plaetze']
        capacity_diff = room_capacity - event_participants
        capacity_differences.append(capacity_diff * event_room_assignment[event_idx, room_idx])

model.Minimize(sum(capacity_differences) + 
               1000 * sum(room_used[room_idx] for room_idx in range(num_rooms)))

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print results

# Print solution status
if status == cp_model.OPTIMAL:
    print('Optimal solution found!')
elif status == cp_model.FEASIBLE:
    print('Feasible solution found, but it may not be optimal.')
else:
    print('No solution found.')
    print(f'Solver status: {status}')
    print('This could mean:')
    print('1. The problem is infeasible (no solution exists that satisfies all constraints)')
    print('2. The solver ran out of time')
    print('3. The solver encountered an error')
    exit()

# Process results if we have a solution
print(f'Number of rooms used: {sum(solver.Value(room_used[room_idx]) for room_idx in range(num_rooms))}')

# Create results dataframe
results = []
for event_idx in range(num_events):
    for room_idx in range(num_rooms):
        if solver.Value(event_room_assignment[event_idx, room_idx]) == 1:
            results.append({
                'veranstaltung_hash_id': veranstaltungen_df.iloc[event_idx]['veranstaltung_hash_id'],
                'raum_id': raeume_df.iloc[room_idx]['raum_id'],
                'veranstaltungsort': raeume_df.iloc[room_idx]['veranstaltungsort'],
                'anzahl_teilnehmer': veranstaltungen_df.iloc[event_idx]['anzahl_teilnehmer'],
                'raum_plaetze': raeume_df.iloc[room_idx]['raum_plaetze'],
                'datum_von': veranstaltungen_df.iloc[event_idx]['datum_von'],
                'datum_bis': veranstaltungen_df.iloc[event_idx]['datum_bis'],
                'lehrgang_ids': veranstaltungen_df.iloc[event_idx]['lehrgang_ids']
            })

results_df = pd.DataFrame(results)
results_df.sort_values(by=['datum_von', 'datum_bis'], inplace=True)
display(results_df)

# Calculate and display statistics
print('\nAssignment Statistics:')
print(f'Total capacity difference: {sum(results_df["raum_plaetze"] - results_df["anzahl_teilnehmer"])}')
print(f'Average capacity difference: {(results_df["raum_plaetze"] - results_df["anzahl_teilnehmer"]).mean():.2f}')
print(f'Number of unique rooms used: {results_df["raum_id"].nunique()}')

Optimal solution found!
Number of rooms used: 8


,veranstaltung_hash_id,raum_id,veranstaltungsort,anzahl_teilnehmer,raum_plaetze,datum_von,datum_bis,lehrgang_ids
1,01bc374a17c881a80bcdb24c71d9497110b4a4ff045d87...,18,A,49,56,2025-05-23 08:30:00,2025-05-23 11:45:00,"[2, 3]"
6,1f8c163b162c8a48b6f0e49be00a07438b04b37f6eaa77...,24,B,64,70,2025-05-23 08:30:00,2025-05-23 11:45:00,[8]
8,3b3b0c621cfd83e5709c8ae573a552d02af0de4de41b6b...,6,A,19,20,2025-05-23 08:30:00,2025-05-23 11:45:00,"[10, 11]"
13,a6e5280de5f082acd14996fa6fa3db2059c58a3aa50b76...,25,B,8,16,2025-05-23 08:30:00,2025-05-23 11:45:00,"[17, 18, 19, 20, 21]"
17,f649b58cb11d7605165fae64fb402a7002f02be2206e66...,27,B,8,10,2025-05-23 08:30:00,2025-05-23 11:45:00,[26]
0,0096055fb3a62972cce6cbdf650358ab14b0a6c38e5667...,27,B,9,10,2025-05-23 12:30:00,2025-05-23 15:45:00,[1]
4,104140207126fb550cb8cdc5d7cefe479994328987685a...,20,A,9,70,2025-05-23 12:30:00,2025-05-23 15:45:00,[6]
7,25a381cb9b5a87f667c211e2eb821a451abb8262ccf192...,5,A,38,40,2025-05-23 12:30:00,2025-05-23 15:45:00,[9]
9,46d2e909e9314495e48e75341e57b5f001d660d9c6db9a...,18,A,49,56,2025-05-23 12:30:00,2025-05-23 15:45:00,[12]
16,c1679d533b7e5dc9c4dfcee9a615072acb1dc1f60db2a2...,6,A,12,20,2025-05-23 12:30:00,2025-05-23 15:45:00,[25]



Assignment Statistics:
Total capacity difference: 256
Average capacity difference: 12.80
Number of unique rooms used: 8
